In [55]:
import ngram
import re
import gensim
import time

from gensim.models import Word2Vec
from nltk import ngrams

import warnings
warnings.filterwarnings(action = 'ignore')

import numpy as np

def backtrace(matrix, line1, line2):
    i = len(line1)
    j = len(line2)
    
    while i > 0 and j > 0:
        if matrix[i][j] == ((matrix[i-1][j-1])+1):
            i -= 1
            j -= 1
        else:
            return

def matrix_filling(matrix, line1, line2, match, mismatch):
    for i in range(1, len(line1)+1):
        for j in range(1, len(line2)+1):
            if line1[j-1] == line2[i-1]:
                a = matrix[i-1][j-1] + match
            else:
                a = matrix[i-1][j-1] + mismatch
            
            b = matrix[i][j-1] + mismatch
            c = matrix[i-1][j] + mismatch
            
            m = max(max(a,b), c)
            
            if m < 0:
                matrix[i][j] = 0
            else:
                matrix[i][j] = m
                
    print(matrix)
    print('"' + line1 + '"')
    print('"' + line2 + '"')

def check(line1, line2):
    if len(line1) == len(line2) and line1 != "\n" and line2 != "\n":
        matrix = np.zeros((len(line1)+1, len(line2)+1), int)
        matrix_filling(matrix, line1, line2, 1, -1)
        
def smith_waterman(text1, text2):
    print("start of ", text1, text2)
    
    file1 = open('texts/' + text1, 'r')
    f1 = file1.readlines()
    file2 = open('texts/' + text2, 'r')
    f2 = file2.readlines()
    for line1 in f1:
        for line2 in f2:
            check(line1, line2)
    print("end of ", text1, text2)

def into_syllable(word, syl_set):
    syllables = []

    bits = ''.join(['1' if l in 'aeiou' else '0' for l in word])

    seperators = (
        ('101', 1),
        ('1001', 2),
        ('10001', 3)
    )

    index, cut_start_pos = 0, 0

    while index < len(bits):

        for seperator_pattern, seperator_cut_pos in seperators:
            if bits[index:].startswith(seperator_pattern):

                syllables.append(word[cut_start_pos:index + seperator_cut_pos])

                index += seperator_cut_pos
                cut_start_pos = index
                break

        index += 1

    syllables.append(word[cut_start_pos:])
    return syllables

def gram1(syl_set,dict_1):
    for syl1 in syl_set:
        for syl2 in syl_set:
            if ngram.NGram.compare(syl1,syl2,N=1) != 0:
                key = syl1+'*'+syl2
                dict_1[key] = ngram.NGram.compare(syl1,syl2,N=1)
                
    for key in dict_1:
        print(key + '\t' + str(dict_1[key]))
            
def gram2(syl_set,dict_2):
    for syl1 in syl_set:
        for syl2 in syl_set:
            for syl3 in syl_set:
                if ngram.NGram.compare(syl1,syl2,syl3,N=2) != 0:
                    key = syl1+'*'+syl2+'*'+syl3
                    dict_2[key] = ngram.NGram.compare(syl1,syl2,syl3,N=2)
                
    for key in dict_2:
        print(key + '\t' + str(dict_2[key]))
        
def gram3(syl_set,dict_3):
    for syl1 in syl_set:
        for syl2 in syl_set:
            for syl3 in syl_set:
                for syl4 in syl_set:
                    if ngram.NGram.compare(syl1,syl2,syl3,syl4,N=3) != 0:
                        key = syl1+'*'+syl2+'*'+syl3+'*'+syl4
                        dict_3[key] = ngram.NGram.compare(syl1,syl2,syl3,syl4,N=3)
                
    for key in dict_3:
        print(key + '\t' + str(dict_3[key]))

def read_file(text):
    syl_set = set([])
    file = open(text, 'r')
    f = file.readlines()
    for line in f:
        for word in re.split('. | ', line):
            for syl in into_syllable(word, syl_set):
                syl_set.add(syl)
                
    print(syl_set)
    
    dict_1 = {}
    dict_2 = {}
    dict_3 = {}
    
    #gram1(syl_set,dict_1)
    #gram2(syl_set,dict_2)
    #gram3(syl_set,dict_3)
    
    n_grams = ngrams(syl_set, 1)
   # for grams in n_grams:
   #     print(grams)
        
   # n_grams = ngrams(syl_set, 2)
   # for grams in n_grams:
     #   print(grams)
        
   # n_grams = ngrams(syl_set, 3)
   # for grams in n_grams:
   #     print(grams)
        
    model1 = gensim.models.Word2Vec(n_grams, min_count = 1, vector_size = 1, window = 5)

    for g1 in ngrams(syl_set, 1):
        for g2 in ngrams(syl_set, 1):
            if len(str(g1)) == len(str(g2)):
                print("Cosine similarity between " + str(g1) + "and" + str(g2) + " - CBOW : ", str(model1.wv.similarity(g1, g2)))
                check(str(g1), str(g2))

    model2 = gensim.models.Word2Vec(n_grams, min_count = 1, vector_size = 1,window = 5, sg = 1)
 
    for g1 in ngrams(syl_set, 1):
        for g2 in ngrams(syl_set, 1):
            if len(str(g1)) == len(str(g2)):
                print("Cosine similarity between " + str(g1) + "and" + str(g2) + " - CBOW : ", str(model2.wv.similarity(g1, g2)))
                check(str(g1), str(g2))
            
read_file("wiki_00.txt")

{'mall', 'ya', 'Ya', 'nu', 'dir', 'sin', 'di', 'kon', 'na', 'to', 'ma', 'du', 'dug', 'Ta', 'red', 'sir', 'yok', 'ol', 'he', "'i", 'lind', 'sind', 'gil', 'gu', 'yu', 'hic', 'gol', 'b', '6', 'Agus', '122', 'gi', 'bi', 'ti', 'ka', 'ko', 'n', 'hind', 'lar', 'il', 'ru', 'den', 'Han', 'mak', 'ul', 'ri', 'ne', 'yi', 'gund', 'lun', 'ke', 'ta', 'rin', 'Gizl', 'da', 'bil', 'sap', 'bu', 'Mo', 'se', 'ih', 'Ha', '2', 'i', 'din', 'hâl', 'nus', 'gor', 'Cen', 'ce', 'tur', 'pi', 'la', 'muzd'}
Cosine similarity between ('mall',)and('mall',) - CBOW :  [[1.]]
[[0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0 1 0 0]
 [0 0 1 3 2 1 0 0 0 0]
 [0 0 0 2 4 3 2 1 0 0]
 [0 0 0 1 3 5 4 3 2 1]
 [0 0 0 0 2 4 6 5 4 3]
 [0 0 1 0 1 3 5 7 6 5]
 [0 0 0 0 0 2 4 6 8 7]
 [0 0 0 0 0 1 3 5 7 9]]
"('mall',)"
"('mall',)"
Cosine similarity between ('mall',)and('lind',) - CBOW :  [[-1.]]
[[0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0 1 0 0]
 [0 0 1 1 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 

Cosine similarity between ('sir',)and('gol',) - CBOW :  [[1.]]
[[0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 1 0 0]
 [0 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 2 1]
 [0 0 0 0 0 0 0 1 3]]
"('sir',)"
"('gol',)"
Cosine similarity between ('sir',)and('122',) - CBOW :  [[1.]]
[[0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 1 0 0]
 [0 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 2 1]
 [0 0 0 0 0 0 0 1 3]]
"('sir',)"
"('122',)"
Cosine similarity between ('sir',)and('lar',) - CBOW :  [[1.]]
[[0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 1 0 0]
 [0 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0]
 [0 0 1 0 0 0 2 1 0]
 [0 0 0 0 0 0 1 3 2]
 [0 0 0 0 0 0 0 2 4]]
"('sir',)"
"('lar',)"
Cosine similarity between ('sir',)and('den',) - CBOW :  [[-1.]]
[[0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 1 0 0]
 [0 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 

Cosine similarity between ('ko',)and('yi',) - CBOW :  [[1.]]
[[0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 2 1 0 1 0 0]
 [0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 1 0 0 1 0 0]
 [0 0 0 0 0 0 2 1]
 [0 0 0 0 0 0 1 3]]
"('ko',)"
"('yi',)"
Cosine similarity between ('ko',)and('ke',) - CBOW :  [[1.]]
[[0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 2 1 0 1 0 0]
 [0 0 1 3 2 1 0 0]
 [0 0 0 2 2 1 0 0]
 [0 0 1 1 1 3 2 1]
 [0 0 0 0 0 2 4 3]
 [0 0 0 0 0 1 3 5]]
"('ko',)"
"('ke',)"
Cosine similarity between ('ko',)and('ta',) - CBOW :  [[1.]]
[[0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 2 1 0 1 0 0]
 [0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 1 0 0 1 0 0]
 [0 0 0 0 0 0 2 1]
 [0 0 0 0 0 0 1 3]]
"('ko',)"
"('ta',)"
Cosine similarity between ('ko',)and('da',) - CBOW :  [[-1.]]
[[0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 2 1 0 1 0 0]
 [0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 1 0 0 1 0 0]
 [0 0 0 0 0 0 2 1]
 [0 0 0 0 0 0 1 3]]
"('ko',)"
"('da',)"
Cosine similarity between ('ko',)and('bu',) - CBOW :  [[-1.]]
[

Cosine similarity between ('sap',)and('yok',) - CBOW :  [[-1.]]
[[0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 1 0 0]
 [0 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 2 1]
 [0 0 0 0 0 0 0 1 3]]
"('sap',)"
"('yok',)"
Cosine similarity between ('sap',)and('gil',) - CBOW :  [[1.]]
[[0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 1 0 0]
 [0 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 2 1]
 [0 0 0 0 0 0 0 1 3]]
"('sap',)"
"('gil',)"
Cosine similarity between ('sap',)and('hic',) - CBOW :  [[-1.]]
[[0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 1 0 0]
 [0 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 2 1]
 [0 0 0 0 0 0 0 1 3]]
"('sap',)"
"('hic',)"
Cosine similarity between ('sap',)and('gol',) - CBOW :  [[1.]]
[[0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 1 0 0]
 [0 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0

Cosine similarity between ('la',)and('ke',) - CBOW :  [[-1.]]
[[0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 2 1 0 1 0 0]
 [0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 1 0 0 1 0 0]
 [0 0 0 0 0 0 2 1]
 [0 0 0 0 0 0 1 3]]
"('la',)"
"('ke',)"
Cosine similarity between ('la',)and('ta',) - CBOW :  [[-1.]]
[[0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 2 1 0 1 0 0]
 [0 0 1 1 0 0 0 0]
 [0 0 0 0 2 1 0 0]
 [0 0 1 0 1 3 2 1]
 [0 0 0 0 0 2 4 3]
 [0 0 0 0 0 1 3 5]]
"('la',)"
"('ta',)"
Cosine similarity between ('la',)and('da',) - CBOW :  [[1.]]
[[0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 2 1 0 1 0 0]
 [0 0 1 1 0 0 0 0]
 [0 0 0 0 2 1 0 0]
 [0 0 1 0 1 3 2 1]
 [0 0 0 0 0 2 4 3]
 [0 0 0 0 0 1 3 5]]
"('la',)"
"('da',)"
Cosine similarity between ('la',)and('bu',) - CBOW :  [[1.]]
[[0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 2 1 0 1 0 0]
 [0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 1 0 0 1 0 0]
 [0 0 0 0 0 0 2 1]
 [0 0 0 0 0 0 1 3]]
"('la',)"
"('bu',)"
Cosine similarity between ('la',)and('Mo',) - CBOW :  [[-1.]]


RuntimeError: you must first build vocabulary before training the model